# 0
### A)

In [1]:
from pathlib import Path
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
env_path = Path("../../code_alongs/11_data_from_API/.env")
load_dotenv(env_path)


API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()



result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [3]:
def get_departures(api):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()


def get_arrivals(api):
    url = f"https://api.resrobot.se/v2.1/arrivalBoard?id=740000002&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()

In [4]:
departures = get_departures(api=API_KEY)
arrivals = get_arrivals(api=API_KEY)

In [5]:
departures.keys(), arrivals.keys()

(dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']),
 dict_keys(['Arrival', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']))

### B)

In [6]:
len(arrivals["Arrival"])

236

### C)

In [7]:
len(departures["Departure"])

246

### D)

In [8]:
departures["Departure"][0].keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [9]:
for depart in departures["Departure"]:
    if "Spårväg" in depart["name"]:
        print(f"{depart['name']}, {depart['direction']}, {depart['time']}")

Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 15:33:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 15:35:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 15:42:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 15:44:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 15:52:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 15:53:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:01:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:03:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:09:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:14:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:17:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:22:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 16:26:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 16:30:00


### E)

In [10]:
# point map över spårvagnar

### F)

# 1
### A)

# Stop Lookup

In [11]:
def get_lookup(search):
    url= f"https://api.resrobot.se/v2.1/location.name?input={search}&format=json&accessId={API_KEY}"
    return requests.get(url).json()

In [12]:
lund= get_lookup("Lund")
lund["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098038'

In [13]:
gothenburg= get_lookup("Göteborg")
gothenburg["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098001'

In [14]:
def route_planner(org, des):
    
    url= f"https://api.resrobot.se/v2.1/trip?format=json&originId={org}&destId={des}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    return requests.get(url).json()

In [15]:
trip= route_planner(740098001, 740098038)

In [16]:
trip.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [17]:
example_trip = trip["Trip"][0]

In [18]:
stops=example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Göteborg Centralstation',
  'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
  'extId': '740000002',
  'routeIdx': 0,
  'lon': 11.973479,
  'lat': 57.708895,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '15:40:00',
  'depDate': '2025-01-09',
  'depDir': 'Köpenhamn Österport (Danmark)',
  'minimumChangeDuration': 'PT15M'},
 {'name': 'Mölndal station',
  'id': 'A=1@O=Mölndal station@X=12018722@Y=57655958@U=1@L=740000315@',
  'extId': '740000315',
  'routeIdx': 1,
  'lon': 12.018722,
  'lat': 57.655958,
  'depTime': '15:48:00',
  'depDate': '2025-01-09',
  'arrTime': '15:48:00',
  'arrDate': '2025-01-09',
  'minimumChangeDuration': 'PT9M'},
 {'name': 'Kungsbacka station',
  'id': 'A=1@O=Kungsbacka station@X=12079614@Y=57490556@U=1@L=740000161@',
  'extId': '740000161',
  'routeIdx': 2,
  'lon': 12.079614,
  'lat': 57.490556,
  'depTime': '15:59:00',
  'depDate': '2025-01-09',
  'arrTime': '15:59:00',
  'arrDate': '2025-01-09',
  'minimumChangeDur

In [19]:
[stop.get("name")for stop in stops]

['Göteborg Centralstation',
 'Mölndal station',
 'Kungsbacka station',
 'Varberg station',
 'Falkenberg station',
 'Halmstad Centralstation',
 'Laholm station',
 'Båstad station',
 'Ängelholm station',
 'Helsingborg Centralstation',
 'Landskrona station',
 'Lund Centralstation']

### B)

In [20]:
sthlm= get_lookup("Stockholm")
sthlm["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098000'

In [21]:
trips = route_planner(740098001, 740098000)
trips

{'Trip': [{'Origin': {'name': 'Göteborg Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
    'extId': '740000002',
    'lon': 11.973479,
    'lat': 57.708895,
    'routeIdx': 0,
    'prognosisType': 'PROGNOSED',
    'time': '15:59:00',
    'date': '2025-01-09',
    'minimumChangeDuration': 'PT15M'},
   'Destination': {'name': 'Stockholm Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
    'extId': '740000001',
    'lon': 18.058151,
    'lat': 59.330136,
    'routeIdx': 18,
    'prognosisType': 'PROGNOSED',
    'time': '20:44:00',
    'date': '2025-01-09',
    'minimumChangeDuration': 'PT20M'},
   'ServiceDays': [{'planningPeriodBegin': '2025-01-02',
     'planningPeriodEnd': '2025-06-15',
     'sDaysR': 'inte varje dag',
     'sDaysI': '9. jan t o m 9. jun 2025 mån - fre; utom 17. t o m 21. apr 2025, 25. apr, 1., 29., 30. maj, 6. jun',
     'sDaysB': 

In [22]:
trip= trips["Trip"][0]

In [23]:
stops= trip["LegList"]["Leg"][0]["Stops"]["Stop"]

In [24]:
[{stop["name"]: stop.get("arrTime")}for stop in stops]

[{'Göteborg Centralstation': None},
 {'Alingsås station': '16:28:00'},
 {'Vårgårda station': '16:40:00'},
 {'Herrljunga station': '16:47:00'},
 {'Falköping Centralstation': '17:03:00'},
 {'Skövde Centralstation': '17:18:00'},
 {'Töreboda station': '17:35:00'},
 {'Laxå station': '17:59:00'},
 {'Hallsberg station': '18:25:00'},
 {'Kumla station': '18:35:00'},
 {'Örebro Södra station': '18:46:00'},
 {'Örebro Centralstation': '18:49:00'},
 {'Arboga station': '19:18:00'},
 {'Köping station': '19:29:00'},
 {'Västerås Centralstation': '19:46:00'},
 {'Enköping station': '20:03:00'},
 {'Bålsta station (Håbo kn)': '20:15:00'},
 {'Sundbyberg station': '20:36:00'},
 {'Stockholm Centralstation': '20:44:00'}]

In [25]:
coords= []
for stop in stops:
    city= get_lookup(stop.get("name"))
    city= city["stopLocationOrCoordLocation"][0]["StopLocation"]
    coords.append((city["lon"], city["lat"]))

In [26]:
coords

[(11.973479, 57.708895),
 (12.532491, 57.926857),
 (12.810285, 58.031204),
 (13.021279, 58.079188),
 (13.553791, 58.176101),
 (13.853195, 58.390898),
 (14.128391, 58.706024),
 (14.616658, 58.989391),
 (15.110391, 59.066698),
 (15.14028, 59.126377),
 (15.202332, 59.269989),
 (15.211348, 59.278942),
 (15.841061, 59.397105),
 (16.004215, 59.506675),
 (16.551712, 59.607705),
 (17.086741, 59.644327),
 (17.532813, 59.568754),
 (17.970938, 59.361032),
 (18.058151, 59.330136)]

In [27]:
data= pd.DataFrame(coords, columns=["lon", "lat"])
data

,lon,lat
0,11.973479,57.708895
1,12.532491,57.926857
2,12.810285,58.031204
3,13.021279,58.079188
4,13.553791,58.176101
5,13.853195,58.390898
6,14.128391,58.706024
7,14.616658,58.989391
8,15.110391,59.066698
9,15.140280,59.126377


In [38]:
fig= px.line_mapbox(data, "lat", "lon", mapbox_style="open-street-map", zoom= 6.5, width= 800)
fig.show()